hi, anthony here

hi laura here

### Loading the data (Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv)

In [7]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = "DATA/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"  
OUTPUT_DIR = "DATA"                    
CITIES = ["New York, NY", "Los Angeles, CA", "Chicago, IL", "San Francisco, CA", "Miami, FL"]

os.makedirs(OUTPUT_DIR, exist_ok=True)
pd.options.display.float_format = "{:,.2f}".format

### Cleans and reshapes the Zillow ZHVI data (wide → long)

This step converts the wide ZHVI CSV into a tidy long table: it detects monthly date columns, coerces values to numeric, melts date columns into `date`/`zhvi` rows, parses `date` values, drops rows with invalid dates, and interpolates small gaps in `zhvi` within each `RegionName`. It also normalizes column types (e.g. `SizeRank`, `RegionType`) and saves the cleaned long dataset to `DATA/zhvi_clean_long.csv`.

In [8]:
def load_zhvi_wide(csv_path: str) -> pd.DataFrame:
    """
    Loads the Zillow ZHVI wide file.
    Assumes metadata columns (RegionID, SizeRank, RegionName, RegionType, StateName)
    followed by hundreds of YYYY-MM-DD monthly columns.
    """
    df = pd.read_csv(csv_path, low_memory=False)
    return df

def zhvi_wide_to_long(df_wide: pd.DataFrame) -> pd.DataFrame:
    """
    Convert wide ZHVI (date columns) to long tidy format: one row per (RegionName, date).
    - Detects date columns by regex
    - Coerces numeric
    - Melts to long
    - Interpolates small gaps within each region
    """
    # Identify date columns like "2000-01-31"
    date_cols = [c for c in df_wide.columns if re.match(r"^\d{4}-\d{2}-\d{2}$", str(c))]
    meta_cols = [c for c in df_wide.columns if c not in date_cols]

    # Coerce values to numeric (leave NaNs if any)
    df_num = df_wide.copy()
    for c in date_cols:
        df_num[c] = pd.to_numeric(df_num[c], errors="coerce")

    # Melt to long
    long = df_num.melt(
        id_vars=meta_cols, value_vars=date_cols,
        var_name="date", value_name="zhvi"
    )
    long["date"] = pd.to_datetime(long["date"], errors="coerce")
    long = long.dropna(subset=["date"]).sort_values(["RegionName", "date"]).reset_index(drop=True)

    # Interpolate small gaps per region
    long["zhvi"] = (
        long.groupby("RegionName", group_keys=False)["zhvi"]
            .apply(lambda s: s.interpolate(limit_direction="both"))
    )

    # Helpful canonical types
    if "SizeRank" in long.columns:
        long["SizeRank"] = pd.to_numeric(long["SizeRank"], errors="coerce")
    if "RegionType" in long.columns:
        long["RegionType"] = long["RegionType"].astype(str)

    return long

# Run it
df_wide = load_zhvi_wide(CSV_PATH)
zhvi_long = zhvi_wide_to_long(df_wide)

# Save a clean copy for reuse
clean_csv_path = os.path.join(OUTPUT_DIR, "zhvi_clean_long.csv")
zhvi_long.to_csv(clean_csv_path, index=False)
print(f"Clean long file saved → {clean_csv_path}")

zhvi_long.head()

Clean long file saved → DATA/zhvi_clean_long.csv


,RegionID,SizeRank,RegionName,RegionType,StateName,date,zhvi
0,394297,677,"Aberdeen, SD",msa,SD,2000-01-31,"132,475.35"
1,394297,677,"Aberdeen, SD",msa,SD,2000-02-29,"132,475.35"
2,394297,677,"Aberdeen, SD",msa,SD,2000-03-31,"132,475.35"
3,394297,677,"Aberdeen, SD",msa,SD,2000-04-30,"132,475.35"
4,394297,677,"Aberdeen, SD",msa,SD,2000-05-31,"132,475.35"
